<a href="https://colab.research.google.com/github/wxjmn2007/LLM-Evaluation-on-Math-Problems/blob/main/Qwen2_5_Math_1.5B_Instruct_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate


In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import torch
import re

model_name = "Qwen/Qwen2.5-Math-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map = "auto"
)

dataset = load_dataset("gsm8k", "main", split = "train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
messages = [{"role": "user", "content": "What is 1 + 1?"}]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to(model.device)


outputs = model.generate(**inputs, max_new_tokens=40)
output_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

print("Model output:\n", output_text)

Model output:
 To solve the problem \(1 + 1\), we can follow these steps:

1. Identify the numbers involved in the addition: 1 and 1.
2. Understand that addition means combining


In [ ]:
dataset = load_dataset("gsm8k", "main", split = "train")

correct = 0
total = 50

def extract_final_answer(text):
    match = re.search(r"####\s*(-?\d+\.?\d*)", text)
    if match:
        return match.group(1).strip()

    numbers = re.findall(r"-?\d+\.?\d*", text)
    return numbers[-1] if numbers else None



for i in range(total):
  question = dataset[i]["question"]
  expected = dataset[i]["answer"]

  messages = [{"role": "user", "content": f"Solve this math problem step by step: \n{question}"}]

  inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
  ).to(model.device)

  outputs = model.generate(**inputs, max_new_tokens = 256)
  output_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens = True)

  predicted = extract_final_answer(output_text)
  is_correct = predicted == expected.split("####")[-1].strip()

  if is_correct:
    correct += 1

  print(f"\n Example {i + 1} ")
  print("Question: ", question)
  print("Expected: ", expected)
  print("Predicted:", predicted)
  print("Correct?", is_correct)
  print("Model output: \n", output_text)

print()
accuracy_per = correct / total * 100
accuracy_per = round(accuracy_per, 1)

print("Accuracy: ", correct, "/", total, "=", str(accuracy_per) + "%")




 Example 1 
Question:  Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Expected:  Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72
Predicted: 72
Correct? True
Model output: 
 To determine the total number of clips Natalia sold in April and May, we need to follow these steps:

1. Identify the number of clips sold in April.
2. Calculate the number of clips sold in May.
3. Add the number of clips sold in April and May to get the total.

First, we know that Natalia sold 48 clips in April. 

Next, we need to find out how many clips she sold in May. According to the problem, she sold half as many clips in May as she did in April. Therefore, we calculate the number of clips sold in May as follows:
\[ \text{Clips sold in May} = \frac{48}{2} = 24 \]

Now, we add the number of clips sold in April and May to find